<a href="https://colab.research.google.com/github/dudeh534/Mirae_Exit/blob/main/KRX_INPUTDATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --target=$my_path bs4 --upgrade

  Preparing metadata (setup.py) ... done
  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached soupsieve-2.5-py3-none-any.whl (36 kB)
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=4b09e8e1b47b8bec45d01082a4b13c9dc3a114971926502211edf1ca95257105
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
!pwd


/content


In [ ]:
#구글드라이브 연동용
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
#os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)

os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

Mounted at /content/drive


In [ ]:
#오라클 연결 확인용
import oracledb
import getpass

#userpwd = getpass.getpass("Enter password: ")

connection=oracledb.connect(
     user="admin",
     password='rlaTPWNS!123',
     dsn="gangnamdatabase_high",
     config_dir="/content/drive/MyDrive/Colab Notebooks",
     wallet_location="/content/drive/MyDrive/Colab Notebooks",
     wallet_password='rlaTPWNS!123')



cursor = connection.cursor()
#sql = "INSERT INTO PD61N001 (ITM_NO, ITM_NM,EXG_MKT_NO) VALUES ('AAPL','애플','101')"
#query = "INSERT INTO table_name(idx,name,title)values(%s,%s,%s)"
#cursor.execute(sql)

#connection.commit()

cursor.execute('select *from pd61n001')
rows = cursor.fetchall()
print(rows)
cursor.close()


[('AAPL', '애플', '101', None), ('A005930\r', '삼성전자', '001', None)]


In [ ]:
#메인코드
import requests as rq
from bs4 import BeautifulSoup
from io import BytesIO
import pandas as pd
import oracledb
import getpass
import re

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

print(biz_day)



gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'

gen_otp_stk = {
    'mktId': 'STK',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = rq.post(down_url, {'code': otp_stk}, headers=headers)
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')

gen_otp_ksq = {
    'mktId': 'KSQ',  # 코스닥 입력
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

down_sector_ksq = rq.post(down_url, {'code': otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)
krx_sector['종목명'] = krx_sector['종목명'].str.strip()
krx_sector['기준일'] = biz_day

#print(krx_sector.head())

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType': '1',
    'mktId': 'ALL',
    'trdDd': biz_day,
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = rq.post(down_url, {'code': otp}, headers=headers)

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
krx_ind['기준일'] = biz_day

#print(krx_ind.head())

kor_ticker = pd.merge(krx_sector,
                      krx_ind,
                      on=krx_sector.columns.intersection(
                          krx_ind.columns).tolist(),
                      how='outer')



import numpy as np
diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))
kor_ticker['종목구분'] = np.where(kor_ticker['종목명'].str.contains('스팩|제[0-9]+호'), '스팩',
                              np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                                       np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠',
                                                np.where(kor_ticker['종목명'].isin(diff),  '기타',
                                                '보통주'))))
kor_ticker = kor_ticker.reset_index(drop=True)
kor_ticker.columns = kor_ticker.columns.str.replace(' ', '')
kor_ticker = kor_ticker[['기준일', '종목코드', '종목명', '시장구분', '종가',
                         '시가총액', 'EPS', '선행EPS', 'BPS', '주당배당금', '종목구분']]
kor_ticker = kor_ticker.replace({np.nan: None})
kor_ticker['기준일'] = pd.to_datetime(kor_ticker['기준일'])

#print(kor_ticker.head())

#오라클 연결
connection=oracledb.connect(
     user="admin",
     password='rlaTPWNS!123',
     dsn="gangnamdatabase_high",
     config_dir="/content/drive/MyDrive/Colab Notebooks",
     wallet_location="/content/drive/MyDrive/Colab Notebooks",
     wallet_password='rlaTPWNS!123')

mycursor = connection.cursor()

args = kor_ticker.values.tolist()


sql = '''INSERT INTO kor_ticker (기준일, 종목코드, 종목명, 시장구분, 종가, 시가총액, EPS, 선행EPS, BPS, 주당배당금, 종목구분) VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11)'''
#query = "INSERT INTO table_name(idx,name,title)values(%s,%s,%s)"
mycursor.executemany(sql,args, batcherrors = False)
#print(args)

connection.commit()

#select_sql = 'SELECT *FROM kor_ticker'
#mycursor.execute(select_sql)

connection.close()

20231114
